In [22]:
from collections import Counter
from scipy import misc

from keras.models import Sequential,Input,Model
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D,Activation, MaxPooling2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM
from keras.utils import to_categorical

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import ngrams

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,accuracy_score,classification_report

import scipy
import copy
import re
import keras
import glob
import numpy as np
import pandas as pd

# import os
# os.environ['KERAS_BACKEND'] = 'theano'

In [2]:
# To download the pre-data if not already installed

# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [3]:
master_data = pd.read_csv("multiclass.csv")
features = np.array(master_data.iloc[:, 1])
labels = np.array(master_data.iloc[:, 0])

In [4]:
# Removed Web links
test = features
p_data = []

for eachPoint in test:
    p_data.append([re.sub(r"(http|ftp|https)\S+", "", str(eachPoint))])

In [5]:
test = p_data

In [6]:
def lowercase(arr):
    return np.char.lower(arr)

In [7]:
def removeStopWords(arr):
    stopWords = set(stopwords.words('english'))
    
    filteredWords = []
    for a in arr:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            if w not in stopWords:
                wordsFiltered = wordsFiltered + " " + w
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [8]:
def removeSymbols(arr):
    symbols = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
    for i in range(len(symbols)):
        arr = np.char.replace(arr, symbols[i], '')   
    return np.char.replace(arr, "  ", " ")

In [9]:
def stemming(word):
    porter_stemmer=PorterStemmer()
    filteredWords = []
    for a in word:
        words = word_tokenize(str(a[0]))
        wordsFiltered = ""
        for w in words:
            wordsFiltered = wordsFiltered + " " + porter_stemmer.stem(w)
        filteredWords.append([wordsFiltered[:len(wordsFiltered)-1]])
    a = np.asarray(filteredWords)
    return np.char.strip(a)

In [10]:
def removeImplicitNoise(arr):
    return np.char.replace (arr, 'NEWLINE_TOKEN', '')

In [11]:
def processClean(data):
    a = removeImplicitNoise(data)
    a = lowercase(a)
    a = removeStopWords(a)
    a = removeSymbols(a)
    a = stemming(a)
    return a

a = processClean(test)

In [12]:
list_l = []

for i in a:
    list_l.append(str(i[0]))

In [13]:
def make_grams(data, n = 2):
    
    grammed_data = []
    
    for i in data:

        k = copy.deepcopy(i)

        for r in range(2,n+1):
            sixgrams = ngrams(i.split(), r)

            for grams in sixgrams:
                g = ""
                for p in grams:
                    g = g+p

                k = k+" "+g
        grammed_data.append(k)
    
    return grammed_data

grammed_data = make_grams(list_l)

In [14]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(feature_vector.shape)

(4500, 81065)


In [15]:
# TF-IDF
vectorizer = TfidfVectorizer()
vectorizer.fit(grammed_data)
vector = vectorizer.transform(grammed_data)

feature_vector = vector.toarray()
print(vector.shape)

(4500, 81065)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, test_size=0.33, random_state=42)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(3015, 81065) (1485, 81065)
(3015,) (1485,)


In [17]:
vocab_size=81065

model=Sequential()

model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(3))
model.add(Activation('softmax'))
model.summary()
 
batch_size=32
epochs=2

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               41505792  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 1539      
__________

In [18]:
data = y_train
print(data)

def encode(data):
    encoded = to_categorical(data,dtype='int')
    return encoded

encoded_data = encode(data)
print(encoded_data)

[2 1 2 ... 2 2 0]
[[0 0 1]
 [0 1 0]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [1 0 0]]


In [19]:
history = model.fit(X_train, encoded_data,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

Epoch 1/2
3015/3015 [==============================] - 6s 2ms/step - loss: 0.7025 - acc: 0.6939
Epoch 2/2
3015/3015 [==============================] - 3s 1ms/step - loss: 0.0690 - acc: 0.9794


In [20]:
test_data=encode(y_test)
score = model.evaluate(X_test, test_data, batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])

1485/1485 [==============================] - 1s 645us/step
Test accuracy: 0.849158249198387


In [23]:
text_labels=encode(y_test)
y_predicted=[]
for i in range(len(X_test)):
    prediction = model.predict(np.array([X_test[i]]))
    predicted_label = np.argmax(prediction[0])
    y_predicted.append(predicted_label)

print(f1_score(y_test,y_predicted,average=None))
print(classification_report(y_test,y_predicted))

[0.81973817 0.86530612 0.86258776]
             precision    recall  f1-score   support

          0       0.85      0.79      0.82       515
          1       0.88      0.85      0.87       499
          2       0.82      0.91      0.86       471

avg / total       0.85      0.85      0.85      1485



In [24]:
confusion_matrix(y_test,y_predicted)

array([[407,  43,  65],
       [ 44, 424,  31],
       [ 27,  14, 430]])